In [1]:
import numpy as np
import pandas as pd

In [2]:
# load datasets
datasets = {}
years = range(2016, 2023)
for year in years:
    print(f"EIE {year} Loading...")
    if int(year) >= 2019:
        file_name = f'Odata{year}File.csv'
    else:
        file_name = f'OpenData{year}.csv'
    try:           
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')
    except:
        dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')
    datasets.update({year:dataset})
    print("success")

EIE 2016 Loading...
success
EIE 2017 Loading...
success
EIE 2018 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:11: DtypeWarning: Columns (119) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2019 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:13: DtypeWarning: Columns (100,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2020 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:13: DtypeWarning: Columns (96,97,98,100,102,103,104,105,116,117,118,120,122,123,124,125) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='Windows 1251')


success
EIE 2021 Loading...


/var/folders/cz/wq9d8j_11fx3b8pjjbk7z8br0000gn/T/ipykernel_75277/3686568105.py:11: DtypeWarning: Columns (117,118,119,121,123,124,125,126,137,138,139,141,143,144,145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f"../../zno/{year}/{file_name}", sep=";", encoding='utf-8')


success
EIE 2022 Loading...
success


In [3]:
for year, dataset in datasets.items():
    #lowercase all columns and add year as attribute
    dataset.columns = [col.lower() for col in dataset.columns]

In [4]:
for year, dataset in datasets.items():
    print([col for col in dataset.columns if col.startswith('eo')])

['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']
['eoname', 'eotypename', 'eoregname', 'eoareaname', 'eotername', 'eoparent']


In [5]:
organizations = pd.DataFrame(columns=['eoparent'])

In [6]:
for year, dataset in datasets.items():
    organizations = pd.concat([organizations, pd.DataFrame(dataset.loc[:, 'eoparent'])], ignore_index=True)


In [7]:
organizations.drop_duplicates(inplace=True)
organizations.reset_index(drop=True, inplace=True)
# when paricipant graduated in the previous years the corresponding information about school
# (everything with 'eo') is NaN
#we drop these rows
organizations.dropna(how = 'all', inplace=True)
organizations

,eoparent
0,Відділ освіти Мелітопольської районної державн...
1,"Управління освіти, молоді та спорту Красилівсь..."
3,Управління освіти Чернівецької міської ради
4,Відділ освіти Дружківської міської ради
5,Управління освіти і науки Тернопільської міськ...
...,...
3795,"Відділ освіти, культури, молоді та спорту Ново..."
3796,"Відділ освіти, культури, молоді та спорту Біло..."
3797,"Відділ освіти, сім’ї, молоді, спорту, культури..."
3798,Гуманітарний відділ Литовезької сільської ради...


In [8]:
organizations.eoparent.unique()

array(['Відділ освіти Мелітопольської районної державної адміністрації Запорізької області',
       'Управління освіти, молоді та спорту Красилівської районної державної адміністрації',
       'Управління освіти Чернівецької міської ради', ...,
       'Відділ освіти, сім’ї, молоді, спорту, культури і туризму Ольгопільської сільської ради Вінницької області',
       'Гуманітарний відділ Литовезької сільської ради Володимир-Волинського району Волинської області',
       'Відділ освіти, культури, туризму, молоді та спорту Присиваської сільської ради'],
      dtype=object)

In [2]:
#!pip install -U duckpy
from duckpy import Client
from tqdm import tqdm
import time
tqdm.pandas()

def search_opendatabot(school_name, max_retry=20):
    """
    Search for school in opendatabot.ua
    """
    client = Client()
    opendatabot_url = ''
    retry = 0
    while not opendatabot_url and retry < max_retry:
        try:
            results = client.search(school_name, limit=20)
        except:
            time.sleep(20)
            continue
        for site in results:
            if site['url'].startswith('https://opendatabot.ua/c/'):
                opendatabot_url = site['url']
                break
        retry += 1
    return {'eoname': school_name, "url": opendatabot_url}

search_opendatabot("Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради")

{'eoname': 'Комаровецький заклад загальної середньої освіти І-ІІ ступенів - заклад дошкільної освіти Барської міської ради',
 'url': 'https://opendatabot.ua/c/26229527'}

In [10]:
opendatabot_links_new = []
for eoname in tqdm(organizations['eoparent']):
    opendatabot_links_new.append(search_opendatabot(eoname))

100%|██████████| 3799/3799 [3:24:08<00:00,  3.22s/it]    


In [12]:
opendatabot_df = pd.DataFrame(opendatabot_links_new)

In [13]:
opendatabot_df

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",
3795,"Відділ освіти, культури, молоді та спорту Біло...",
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",
3797,Гуманітарний відділ Литовезької сільської ради...,


In [14]:
opendatabot_df[opendatabot_df.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3781,"Відділ освіти, культури, молоді та спорту Нижн...",https://opendatabot.ua/c/44041491
3782,"Відділ освіти, культури, спорту та туризму Рай...",https://opendatabot.ua/c/43915446
3783,Гуманітарний відділ виконавчого комітету Узинс...,https://opendatabot.ua/c/41973438
3784,Шабівська сільська рада,https://opendatabot.ua/c/04378043


In [15]:
opendatabot_df.to_csv('organizations_link.csv', index=False)

# Next try

In [129]:
unmatched_link = pd.read_csv('organizations_link.csv', dtype=str)
unmatched_link

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343


In [130]:
unmatched_new = unmatched_link[~unmatched_link.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]
unmatched_new

,eoname,url
62,Відділ освіти Броварської райдержадміністрації,NaN
63,"Управління освіти, молоді та спорту Дубровицьк...",NaN
74,Відділ освіти виконкому Дзержинської районної ...,NaN
85,Відділ освіти Городнянської районної державної...,NaN
86,Управління освіти і науки Чернігівської обласн...,NaN
...,...,...
3663,"Управління освіти, культури, туризму, молоді т...",NaN
3664,Відділ освіти Гребінківської селищної ради,NaN
3665,Відділ освіти Зміївської міської ради Чугуївсь...,NaN
3666,"Відділ освіти, культури, молоді та спорту Мар'...",NaN


In [131]:
opendatabot_links_new = []
for eoname in tqdm(unmatched_new['eoname']):
    opendatabot_links_new.append(search_opendatabot(eoname))

100%|██████████| 173/173 [12:26<00:00,  4.31s/it]


In [132]:
opendatabot_df_new = pd.DataFrame(opendatabot_links_new)
opendatabot_df_new

,eoname,url
0,Відділ освіти Броварської райдержадміністрації,
1,"Управління освіти, молоді та спорту Дубровицьк...",
2,Відділ освіти виконкому Дзержинської районної ...,
3,Відділ освіти Городнянської районної державної...,
4,Управління освіти і науки Чернігівської обласн...,
...,...,...
168,"Управління освіти, культури, туризму, молоді т...",
169,Відділ освіти Гребінківської селищної ради,
170,Відділ освіти Зміївської міської ради Чугуївсь...,
171,"Відділ освіти, культури, молоді та спорту Мар'...",


In [133]:
opendatabot_df_new[opendatabot_df_new.url!='']

,eoname,url
28,"Управління освіти, науки, молоді та спорту Кір...",https://opendatabot.ua/c/02143927
29,Відділ освіти Солонянської райдержадміністраці...,https://opendatabot.ua/c/02142543
30,Відділ освіти Бердянської районної державної а...,https://opendatabot.ua/c/02136169
44,Відділ освіти Броварської районної державної а...,https://opendatabot.ua/c/02143873
45,"Управління освіти, молоді та спорту Володимире...",https://opendatabot.ua/c/02145820
46,"Відділ освіти, молоді та спорту Уманської райо...",https://opendatabot.ua/c/02146959
47,Управління освіти адміністрації Холодногірсько...,https://opendatabot.ua/c/02146305
61,Відділ освіти Кременецької міської ради,https://opendatabot.ua/c/40210966
62,"Відділ освіти, культури, молоді та спорту Корн...",https://opendatabot.ua/c/41090331
63,"Відділ освіти, охорони здоров'я, культури, мол...",https://opendatabot.ua/c/02143146


In [134]:
df = unmatched_link.merge(opendatabot_df_new[['url']], left_on=[unmatched_link['eoname']], right_on=[opendatabot_df_new['eoname']], how='left')

df[~df.url_y.isnull()]

,key_0,eoname,url_x,url_y
62,Відділ освіти Броварської райдержадміністрації,Відділ освіти Броварської райдержадміністрації,NaN,
63,"Управління освіти, молоді та спорту Дубровицьк...","Управління освіти, молоді та спорту Дубровицьк...",NaN,
74,Відділ освіти виконкому Дзержинської районної ...,Відділ освіти виконкому Дзержинської районної ...,NaN,
85,Відділ освіти Городнянської районної державної...,Відділ освіти Городнянської районної державної...,NaN,
86,Управління освіти і науки Чернігівської обласн...,Управління освіти і науки Чернігівської обласн...,NaN,
...,...,...,...,...
3663,"Управління освіти, культури, туризму, молоді т...","Управління освіти, культури, туризму, молоді т...",NaN,
3664,Відділ освіти Гребінківської селищної ради,Відділ освіти Гребінківської селищної ради,NaN,
3665,Відділ освіти Зміївської міської ради Чугуївсь...,Відділ освіти Зміївської міської ради Чугуївсь...,NaN,
3666,"Відділ освіти, культури, молоді та спорту Мар'...","Відділ освіти, культури, молоді та спорту Мар'...",NaN,


In [135]:
unmatched_link = unmatched_link.merge(opendatabot_df_new[['url']], left_on=[unmatched_link['eoname']], right_on=[opendatabot_df_new['eoname']], how='left')
unmatched_link = unmatched_link.drop(columns = ['key_0'], axis=1)
unmatched_link.head()

,eoname,url_x,url_y
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100,NaN
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095,NaN
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345,NaN
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848,NaN
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169,NaN


In [136]:
unmatched_link.loc[unmatched_link['url_x'].isna(), 'url_x'] = ''

In [137]:
unmatched_link.loc[unmatched_link['url_x'] == '', 'url_x'] = unmatched_link.loc[unmatched_link['url_x'] == '', 'url_y']
unmatched_link

,eoname,url_x,url_y
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100,NaN
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095,NaN
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345,NaN
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848,NaN
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169,NaN
...,...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592,NaN
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359,NaN
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142,NaN
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343,NaN


In [138]:
unmatched_link = unmatched_link.drop(columns=['url_y'], axis=1)
unmatched_link = unmatched_link.rename(columns = {'url_x':'url'})
unmatched_link

,eoname,url
0,Відділ освіти Мелітопольської районної державн...,https://opendatabot.ua/c/02136100
1,"Управління освіти, молоді та спорту Красилівсь...",https://opendatabot.ua/c/38418095
2,Управління освіти Чернівецької міської ради,https://opendatabot.ua/c/02147345
3,Відділ освіти Дружківської міської ради,https://opendatabot.ua/c/02142848
4,Управління освіти і науки Тернопільської міськ...,https://opendatabot.ua/c/26198169
...,...,...
3794,"Відділ освіти, культури, молоді та спорту Ново...",https://opendatabot.ua/c/43981592
3795,"Відділ освіти, культури, молоді та спорту Біло...",https://opendatabot.ua/c/41555359
3796,"Відділ освіти, сім’ї, молоді, спорту, культури...",https://opendatabot.ua/c/44072142
3797,Гуманітарний відділ Литовезької сільської ради...,https://opendatabot.ua/c/44545343


In [139]:
unmatched_link.drop_duplicates(inplace=True)

In [140]:
unmatched_link.reset_index(drop=True, inplace=True)

In [141]:
unmatched_link[~unmatched_link.url.apply(lambda x: isinstance(x, str) and len(x.strip())> 0)]

,eoname,url
62,Відділ освіти Броварської райдержадміністрації,
63,"Управління освіти, молоді та спорту Дубровицьк...",
74,Відділ освіти виконкому Дзержинської районної ...,
85,Відділ освіти Городнянської районної державної...,
86,Управління освіти і науки Чернігівської обласн...,
...,...,...
3663,"Управління освіти, культури, туризму, молоді т...",
3664,Відділ освіти Гребінківської селищної ради,
3665,Відділ освіти Зміївської міської ради Чугуївсь...,
3666,"Відділ освіти, культури, молоді та спорту Мар'...",


In [142]:
unmatched_link.to_csv('organizations_link.csv',  index=False, encoding='utf-8')